<a href="https://colab.research.google.com/github/Namesakenberg/machine_learning/blob/main/bagging_regressors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/content/BostonHousing.csv')

In [4]:
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     506 non-null    float64
 1   zn       506 non-null    float64
 2   indus    506 non-null    float64
 3   chas     506 non-null    int64  
 4   nox      506 non-null    float64
 5   rm       506 non-null    float64
 6   age      506 non-null    float64
 7   dis      506 non-null    float64
 8   rad      506 non-null    int64  
 9   tax      506 non-null    int64  
 10  ptratio  506 non-null    float64
 11  b        506 non-null    float64
 12  lstat    506 non-null    float64
 13  medv     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


In [6]:
df.shape

(506, 14)

In [7]:
X = df.drop(columns=['medv'])
y = df['medv']

In [8]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=123)

In [9]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(404, 13) (102, 13) (404,) (102,)


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
lr = LinearRegression()
knr = KNeighborsRegressor()
dtr = DecisionTreeRegressor()

lr.fit(X_train,y_train)
knr.fit(X_train,y_train)
dtr.fit(X_train,y_train)

y_pred1 = lr.predict(X_test)
y_pred2 = knr.predict(X_test)
y_pred3 = dtr.predict(X_test)

print("r2_lin_reg ",r2_score(y_test,y_pred1))
print("r2_dtr",r2_score(y_test,y_pred3))
print("r2_KNR ",r2_score(y_test,y_pred2))



r2_lin_reg  0.6592466510354096
r2_dtr 0.46622646270348334
r2_KNR  0.5475962186976784


In [11]:
from sklearn.ensemble import BaggingRegressor
bag_reg = BaggingRegressor(random_state=1)
bag_reg.fit(X_train,y_train)
y_pred = bag_reg.predict(X_test)

In [13]:
print("training r2",bag_reg.score(X_train , y_train))
print("testing r2",r2_score(y_test,y_pred))

training r2 0.9799359879973576
testing r2 0.8184644795411804


In [14]:
from sklearn.model_selection import GridSearchCV
params={
    'estimator':[None,LinearRegression(),KNeighborsRegressor()],
    'n_estimators':[20,50,100],
    'max_samples':[0.5,1.0],
    'max_features':[0.5,1.0],
    'bootstrap':[True,False],
    'bootstrap_features':[True,False]
}

bagging_regressor_grid = GridSearchCV(BaggingRegressor(random_state=1,n_jobs=-1),param_grid=params,cv=3,n_jobs=-1,verbose=1)
bagging_regressor_grid.fit(X_train,y_train)
y_pred = bagging_regressor_grid.predict(X_test)

print("training r2 : ",bagging_regressor_grid.score(X_train , y_train))
print("testing r2 : " ,bagging_regressor_grid.score(X_test,y_test))
print("best_r2_gridsearchCV : ",bagging_regressor_grid.best_score_)
print("best_parmeters : ",bagging_regressor_grid.best_params_)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
training r2 :  0.9832238020638931
testing r2 :  0.805126651796481
best_r2_gridsearchCV :  0.8713456857539185
best_parmeters :  {'bootstrap': True, 'bootstrap_features': False, 'estimator': None, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 50}
